In [1]:
%reload_ext autoreload
%autoreload 2

In [4]:
import copy
import time
import warnings

import matplotlib.pyplot as plt
import numpy as np

# Suppress common UMAP and sklearn warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# 🌌 <font color="grey"> Local2Global X - Graph Representation Learning at Scale</font>

#### <font color="grey">  Table of Contents</font>

🏗️ <a href='#chapter1'>Structure</a>

📊 <a href='#chapter2'>Datasets</a>

🌐 <a href='#chapter3'>Graphs</a>

🧩 <a href='#chapter4'>Patches</a>

🎯 <a href='#chapter5'>Embedding</a>

🔗 <a href='#chapter6'>Alignment</a>

🌳 <a href='#chapter7'>Hierarchical alignment</a>

📈 <a href='#chapter8'>Visualisation</a>

###  <a id='chapter1'> 🏗️ <font color="grey">Structure </font></a>

There are five main parts to the package, organised as follows.

```
l2gv2/
├── datasets/
├── graphs/
├── patch/
├── embedding/
└── align/
    ├── l2g/
    └── geo/
```

A brief overview of the contents:

* ```datasets``` contains interfaces are provided for various common benchmark datasets. 
* ```graphs``` contains wrappers for graphs represented as lists of edges in pytorch-geometric ```data.edge_index``` format. These implemented features such as fast adjacency look-up and a variety of algorithms on graphs.
* ```patch``` directory contains datastructures to represent patches and patch graphs, as well as methods to subdivide a graph into patches. 
* ```embedding``` contains various graph embedding methods, including Graph Autoencoders (GAE) and [Variational Graph Autoencoders (VGAE)](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.models.VGAE.html).
* ```align``` contains two methods to compute the alignment of patches into a single graph embedding: eigenvalue synchronisation based on the [Local2Global](https://link.springer.com/article/10.1007/s10994-022-06285-7) algorithm, and the new method based on learning the alignment using a one-layer neural network.

###  <a id='chapter2'> 📊 <font color="grey">Datasets </font></a>

The L2GX framework provides access to multiple graph datasets spanning different domains and scales. All datasets are accessible through the unified `get_dataset()` interface and support conversion between multiple formats (PyTorch Geometric, Raphtory, Polars). In addition, all standard datasets available in PyTorch Geometric can be used.

| Dataset | Type | Nodes | Edges | Features | Domain |
|---------|------|-------|-------|----------|--------|
| **Cora** | Static Citation | 2,708 | 10,556 | 1,433 | 📚 Academic Papers |
| **CiteSeer** | Static Citation | 3,327 | 9,104 | 3,703 | 📚 Academic Papers |
| **PubMed** | Static Citation | 19,717 | 88,648 | 500 | 📚 Academic Papers |
| **AS-733** | Temporal Network | 7,716 | 45,645 | Temporal | 🌐 Internet Infrastructure |
| **DGraph** | Financial | ~3M | ~4M | Multiple | 💰 Fraud Detection |
| **Elliptic** | Bitcoin | 203,769 | 234,355 | 166 | ₿ Cryptocurrency |
| **Elliptic2** | Bitcoin | 42M | 196M | 122K Subgraphs | ₿ Cryptocurrency |
| **MAG240M** | Academic | 244M+ | 1.7B+ | Rich | 🎓 Citation Graph |
| **ORBITAAL** | Bitcoin Temporal | 252M (1K sample) | 785M (5K sample) | Temporal + Anomaly | ₿ Financial Fraud |
| **BTCGraph** | Bitcoin Temporal | 252M (100K sample) | 785M | Entity Type | ₿ Cryptocurrency |


#### Dataset Details

* **Cora**: The [Cora dataset](https://graphsandnetworks.com/the-cora-dataset/) is a citation network of 2,708 scientific publications divided into 7 classes. Each node has a 1,433-dimensional feature vector indicating word presence/absence. Accessed through PyTorch Geometric's [Planetoid](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.datasets.Planetoid.html) dataset.

* **CiteSeer**: The [CiteSeer dataset](https://citeseerx.ist.psu.edu/) is a citation network of 3,327 scientific publications classified into 6 classes (Agents, AI, DB, IR, ML, HCI). Each node has a 3,703-dimensional feature vector. Part of the Planetoid collection, this dataset provides a slightly larger and more challenging benchmark than Cora.

* **PubMed**: The [PubMed dataset](https://pubmed.ncbi.nlm.nih.gov/) is the largest Planetoid dataset with 19,717 scientific publications from the PubMed database classified into 3 diabetes-related classes. Each node has a 500-dimensional TF-IDF feature vector derived from paper abstracts. Excellent for testing scalability of graph neural network methods.

* **AS-733**: The [SNAP autonomous systems AS-733](https://snap.stanford.edu/data/as-733.html) dataset contains 733 daily snapshots spanning 785 days (November 1997 to January 2000). Nodes represent autonomous systems and edges indicate communication events.

* **DGraph**: [DGraph](https://dgraph.xinye.com/dataset) is a real-world financial graph for anomaly detection research. Described in [DGraph: A Large-Scale Financial Dataset for Graph Anomaly Detection](https://arxiv.org/abs/2207.03579). Requires manual download.

* **Elliptic**: The [Elliptic dataset](https://www.kaggle.com/datasets/ellipticco/elliptic-data-set) maps Bitcoin transactions to licit/illicit categories. Contains 203,769 transactions with 166 features each. Used in [Anti-Money Laundering in Bitcoin](https://arxiv.org/pdf/1908.02591) research. Requires manual download from Kaggle.

* **Elliptic2**: The [Elliptic 2 dataset](https://arxiv.org/abs/2404.19109) dataset consists of 49M nodes and 196M edges, representing transactactions. The graph is geared towards subgraph classification, containing 122K labelled subgraphs of bitcoin clusters.

* **MAG240M**: The [MAG240M](https://ogb.stanford.edu/docs/lsc/mag240m/) dataset is a large heterogeneous academic citation graph with 244+ million nodes (papers, authors, institutions, fields) and 1.7+ billion edges. Requires the OGB library and substantial storage (~100GB).

* **ORBITAAL**: The [ORBITAAL](https://www.nature.com/articles/s41597-025-04595-8) dataset is a comprehensive temporal Bitcoin transaction graph covering 13 years (2009-2021) with 252M entities and 785M transactions. Features timestamped transactions, entity types (exchanges, wallets, services, miners), and anomaly labels for financial fraud detection. Ideal for temporal graph neural networks and cryptocurrency flow analysis.

* **BTCGraph**: The [Bitcoin Large Scale Transaction Graph](https://www.nature.com/articles/s41597-025-04684-8) is a large-scale, temporally annotated graph dataset representing Bitcoin transactions. The dataset comprises 252 million nodes and 785 million edges, with each node and edge timestamped for temporal analysis. The dataset contains two labeled subsets: 34,000 nodes annotated with entity types, and 100,000 Bitcoin addresses labeled with entity names and types.


For the datasets requiring manual download, provide the path:
```
elliptic = get_dataset("Elliptic", source_file="/path/to/elliptic.zip")
dgraph = get_dataset("DGraph", source_file="/path/to/dgraph.zip")
```

All datasets support conversion to different formats and follow the PyTorch Geometric convention. Temporal graphs return iterables over time slices, and graphs can be exported to Raphtory or NetworkX formats for analysis.

Internally, the graphs are stored in parquet format, with the edge data and node features available as polars dataframes.

In [5]:
from l2gx.datasets import get_dataset, list_available_datasets

datasets = list_available_datasets()
print(f"Current datasets: {datasets}")

Current datasets: ['as-733', 'Cora', 'CiteSeer', 'PubMed', 'DGraph', 'Elliptic', 'MAG240M', 'ORBITAAL', 'Bitcoin', 'BTC', 'btc']


In [ ]:
cora = get_dataset("Cora")
tg_cora = cora.to("torch-geometric")
print(tg_cora)

In [ ]:
G = cora.to("networkx")
labels = tg_cora.y.numpy()
print(f"Graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
print(f"Labels: {len(np.unique(labels))} unique classes")

In [1]:
import polars as pl

In [2]:
df = pl.read_parquet("../data/BTCGraph/node_features.parquet")
df.head()

alias,degree,degree_in,degree_out,total_transactions_in,total_transactions_out,min_sent,max_sent,total_sent,min_received,max_received,total_received,cluster_size,first_transaction_in,last_transaction_in,first_transaction_out,last_transaction_out,cluster_num_edges,cluster_num_cc,cluster_num_nodes_in_cc,label
i64,i32,i32,i32,i32,i32,i64,i64,i64,i64,i64,i64,i32,i32,i32,i32,i32,i32,i32,i32,str
1,920,920,null,3030,null,null,null,null,1,400000000,1852174811,null,123723,699999,null,null,null,null,null,"""INDIVIDUAL"""
2,118,118,null,132,null,null,null,null,1,22927472,34736774,1,127659,698505,null,null,null,null,null,null
3,31,31,null,32,null,null,null,null,1,500000,1580106,1,204814,676543,null,null,null,null,null,null
4,9,9,null,9,null,null,null,null,547,500000,533182,1,254137,666545,null,null,null,null,null,null
5,9,9,null,9,null,null,null,null,547,500000,572448,1,307383,666545,null,null,null,null,null,null


In [6]:
df.shape

(252219007, 21)

In [10]:
sdf = df[pl.col('label').is_null()]

TypeError: cannot select columns using key of type 'Expr': <Expr ['col("label").is_null()'] at 0x106468D00>

In [7]:
edf = pl.read_parquet("../data/BTCGraph/transaction_edges.parquet")

In [3]:
# Load manageable sample 
btc = get_dataset("btc", max_nodes=10000)

# Get the data
data = btc[0]
print(f"Nodes: {data.num_nodes}, Features: {data.x.shape}")
print(f"Classes: {data.num_classes}")
print(f"Labels: {data.label_names}")

NameError: name 'get_dataset' is not defined

In [9]:
edf.head()

a,b,reveal,last_seen,total,min_sent,max_sent,total_sent
i32,i32,i32,i32,i32,i64,i64,i64
10,769,170,170,1,1000000000,1000000000,1000000000
10,184,181,181,1,1000000000,1000000000,1000000000
10,186,182,182,1,100000000,100000000,100000000
10,188,183,183,1,100000000,100000000,100000000
10,259,248,248,1,1000000000,1000000000,1000000000


In [ ]:
# Temporal data
# as733 = get_dataset("as-733")

###  <a id='chapter3'> 🌐 <font color="grey">Graphs </font></a>

```TGraph``` is a wrapper for torch-geometric ```Data``` objects. These include, among other things, methods for fast adjacency look-up and various optimizations. These are mostly used when performing graph clustering and generating patches.

In [ ]:
from l2gx.graphs import TGraph

In [ ]:
tg = TGraph(cora[0].edge_index, edge_attr=cora[0].edge_attr, x=cora[0].x)
print(tg.adj_index)
print(tg.x)

In a future iteration one can think about consolidating this part by having graphs represented in some existing graph package like Raphtory.

###  <a id='chapter4'> 🧩 <font color="grey">Patches </font></a>

A patch can equivalently refer to a subgraph or to an embedding of this subgraph. As a set of points, a patch is represented using the ```Patch``` class. A ```Patch``` object has the properties ```nodes```, ```index``` and ```coordinates```. ```nodes``` is simply a list of the nodes from the original graph that are present in the patch. ```index``` is a dict that maps each node to an index into ```coordinates```, which is just a list of coordinates. For example, if a graph embedding consists of four nodes in two dimensions as follows, and a patch is represented by the solid circles, then the corresponding object would have the following properties:

![Patch](./images/square_patch.png)


In [ ]:
from l2gx.patch.patches import Patch

In [ ]:
p = Patch([0, 2, 3], np.array([[0.0, 0.0], [1.0, 0.0], [1.0, 1.0]]))
print(f"Coordinates: {p.coordinates[0]}, {p.coordinates[1]}, {p.coordinates[2]}")
print(f"Nodes (with noden numbering from original graph): {p.nodes}")
print(f"Index dictionary: {p.index}")

In [ ]:
from l2gx.patch import create_patches
from run.plots import plot_patch_graph

In [ ]:
graph = TGraph(tg_cora.edge_index, edge_attr=tg_cora.edge_attr, x=tg_cora.x)
patch_graph = create_patches(graph, num_patches=10, clustering_method="metis")
patches = patch_graph.patches
overlap_nodes = patch_graph.overlap_nodes

The patches from the nodes of a **patch graph**, where two nodes are connected by an edge if the patches contain overlapping nodes. The alignment tasks consists of making the correponding coordinates overlap as much as possible.

In [ ]:
print([len(p.nodes) for p in patches])

In [ ]:
fig = plot_patch_graph(patch_graph, tg, filename=None)
fig.show()

###  <a id='chapter5'> 🎯 <font color="grey">Embedding </font></a>

The L2GX framework implements several graph embedding methods: ```SVDEmbedding```, ```GAEEmbedding```, ```VGAEEmbedding```, ```GraphSAGEEmbedding``` and ```DGIEmbedding```. The first three are based on transductive learning, while the last two are inductive.

* <font color="grey">SVD</font> - Classical spectral approach using eigendecomposition
* <font color="grey">GAE</font> - [Graph Auto-Encoder ](https://arxiv.org/abs/1611.07308) for deterministic reconstruction
* <font color="grey">VGAE</font> - [Variational Graph Auto-Encoder](https://arxiv.org/abs/1611.07308) with probabilistic latent variables
* <font color="grey">GraphSAGE</font> - [Inductive Representation Learning on Large Graphs](https://arxiv.org/abs/1706.02216) for scalable embedding
* <font color="grey">DGI</font> - [Deep Graph Infomax](https://arxiv.org/abs/1809.10341) using self-supervised contrastive learning

All methods are accessible through a unified interface with the ```get_embedding()``` function and registry system. The demonstration below shows convergence analysis, quality metrics, and UMAP visualizations for comprehensive comparison.

In [ ]:
from l2gx.embedding import get_embedding

In [ ]:
start_time = time.time()
embedder = get_embedding("vgae", embedding_dim=64, epochs=200)
embedding = embedder.fit_transform(tg_cora, verbose=False)
end_time = time.time()
embedding_time = end_time - start_time
loss_history = embedder.get_training_history()

print(f"VGAE embedding completed in {embedding_time:.2f}s")
print(f"   Embedding shape: {embedding.shape}")

In [ ]:
plt.plot(loss_history['epochs'], loss_history['losses'])
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("VGAE Training Loss Curve")

plt.show()

###  <a id='chapter6'> 🔗 <font color="grey">Alignment </font></a>

Two general alignment methods are available: ```l2g``` and ```geo```. The ```l2g``` method is essentially the original Local2Global approach, though with various options for speeding up the eigendecomposition using randomized linear algebra. The ```geo``` method is based on optimization on the orthogonal group.

In [ ]:
from example import (
    generate_patch_graph,
    generate_points,
    plot_patches,
    transform_patches,
)

from l2gx.align import get_aligner, procrustes_error

rg = np.random.default_rng()

We test the alignment method using a synthetic point cloud in two dimensions and in 256 dimensions.

In [ ]:
points = generate_points(
    n_clusters=5, scale=1.0, std=0.2, max_size=2000, min_size=128, dim=2
)
pg = generate_patch_graph(
    points,
    sample_size=10,
    min_degree=4,
    min_overlap=64,
    min_patch_size=128,
    eps=1,
    kmeans=False,
)
patches = pg.patches

The patches are moved around, scaled and noise is added.

In [ ]:
noise_level = 0.1
shift_scale = 10
scale_range = [0.01, 100]

# Create transformed copies of the patches
transformed_patches = [copy.deepcopy(p) for p in patches]

# Add noise to the transformed patches
if noise_level > 0:
    for patch in transformed_patches:
        noise = rg.normal(loc=0, scale=noise_level, size=patch.coordinates.shape)
        patch.coordinates += noise

transformed_patches = transform_patches(
    transformed_patches, shift_scale=shift_scale, scale_range=scale_range
)
transformed_pg = copy.deepcopy(pg)
transformed_pg.patches = transformed_patches

In [ ]:
plot_patches(patches, transformed_patches)

In [ ]:
l2g_aligner = get_aligner("l2g", randomized_method="randomized", sketch_method="rademacher")
l2g_aligner.align_patches(transformed_pg)
embedding = l2g_aligner.get_aligned_embedding()
error = procrustes_error(embedding, points)
print(f"Procrustes error: {error}")

In [ ]:
plot_patches(patches, l2g_aligner.patches)

In [ ]:
# High-dimensional patches
points = generate_points(
    n_clusters=5, scale=1.0, std=0.2, max_size=2000, min_size=128, dim=256
)
pg = generate_patch_graph(
    points,
    sample_size=20,
    min_degree=4,
    min_overlap=64,
    min_patch_size=128,
    eps=1,
    kmeans=False,
)
patches = pg.patches
overlap_nodes = pg.overlap_nodes

In [ ]:
noise_level = 0.1
shift_scale = 10
scale_range = [0.01, 100]

# Create transformed copies of the patches
transformed_patches = [copy.deepcopy(p) for p in patches]

# Add noise to the transformed patches
if noise_level > 0:
    for patch in transformed_patches:
        noise = rg.normal(loc=0, scale=noise_level, size=patch.coordinates.shape)
        patch.coordinates += noise

transformed_patches = transform_patches(
    transformed_patches, shift_scale=shift_scale, scale_range=scale_range
)
transformed_pg = copy.deepcopy(pg)
transformed_pg.patches = transformed_patches

In [ ]:
l2g_aligner_standard = get_aligner("l2g", randomized_method="standard")
time_start = time.time()
l2g_aligner_standard.align_patches(transformed_pg)
time_end = time.time()
print(f"Time taken: {time_end - time_start} seconds")
embedding_standard = l2g_aligner_standard.get_aligned_embedding()

l2g_aligner_randomized = get_aligner(
    "l2g", randomized_method="randomized", sketch_method="rademacher"
)
time_start = time.time()
l2g_aligner_randomized.align_patches(transformed_pg)
time_end = time.time()
print(f"Time taken: {time_end - time_start} seconds")
embedding_randomized = (
    l2g_aligner_randomized.get_aligned_embedding()
)

procrustes_error_standard = procrustes_error(embedding_standard, points)
procrustes_error_randomized = procrustes_error(embedding_randomized, points)

print(f"Procrustes error (standard): {procrustes_error_standard}")
print(f"Procrustes error (randomized): {procrustes_error_randomized}")

In [ ]:
from run.random_graph.roc import RandomOverlappingCommunities
from run.random_graph.patch_integration import CommunityToPatchConverter

In [ ]:
roc = RandomOverlappingCommunities(
    n_nodes=200,
    n_communities=8,
    community_generation="preferential",
    average_community_size=25,
    overlap_factor=0.3,
    membership_strength_dist="beta",
    base_edge_prob=0.15,
    random_state=42,
)

G = roc.generate_graph()

In [ ]:
converter = CommunityToPatchConverter(min_patch_size=10)
patches = converter.communities_to_patches(graph, community_nodes, embeddings)
patch_graph = converter.create_patch_graph(patches)

aligner = get_aligner("l2g")
aligner.align_patches(patch_graph)

###  <a id='chapter7'> <font color="grey">7. Hierarchical alignment </font></a>

To be done.